# 📄 SOW Generator (Google Colab Version)

Generates a Statement of Work (SOW) PDF from Meeting Minutes using OpenAI.

**Instructions:**
1.  Add your API Key to Colab Secrets (`LLM_API_KEY`) or enter it when prompted.
2.  Upload `SOW-TEMPLATE.pdf` to the Files section (sidebar).
3.  Run the cells below step-by-step.

> **Note on GPU:** This notebook is configured to use a GPU Runtime. However, since the current logic uses the OpenAI API (cloud-based), the local GPU will not significantly speed up the LLM generation. If you modify the code to use a local model (e.g., Llama-3 via Hugging Face), the GPU will be utilized.

In [ ]:
# @title 1. Install Dependencies
# @markdown Run this cell to install required libraries.
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("Installing dependencies...")
try:
    import openai
    import pypdf
    import xhtml2pdf
except ImportError:
    install("openai")
    install("pypdf")
    install("xhtml2pdf")
    print("Dependencies installed!")

In [ ]:
# @title 2. Core Logic & Imports
import os
from io import BytesIO
from openai import OpenAI
from pypdf import PdfReader
from xhtml2pdf import pisa
import time

# --- CORE FUNCTIONS (Ported from sow_generator.py) ---

def extract_text_from_pdf(pdf_path):
    """Extracts text content from a PDF file."""
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF template: {str(e)}"

def generate_pdf_from_html(html_content):
    """Converts HTML string to PDF bytes."""
    result = BytesIO()
    pisa.CreatePDF(BytesIO(html_content.encode('utf-8')), result)
    return result.getvalue()

def chunk_text(text, size=2000):
    return [text[i:i+size] for i in range(0, len(text), size)]

def generate_sow_draft(mom_text, client, model):
    """Generates a text/markdown draft of the SOW from MOM details."""
    
    # 1. Summarize MOM if too large
    mom_chunks = chunk_text(mom_text)
    consolidated_info = ""
    
    if len(mom_chunks) > 1:
        print(f"[+] Processing MOM in {len(mom_chunks)} batches...")
        for i, chunk in enumerate(mom_chunks, 1):
            print(f"    - Processing Batch {i}/{len(mom_chunks)}...")
            summary_prompt = (
                f"I have a section of Meeting Minutes. Extract key project details (Scope, Timeline, Budget, Team, Deliverables, etc.) as concise bullet points.\n"
                f"Ignore conversational filler.\n\n"
                f"MOM Segment:\n"
                f"{chunk}"
            )
            try:
                resp = client.chat.completions.create(
                    model=model,
                    messages=[{"role": "user", "content": summary_prompt}],
                    temperature=0.3
                )
                consolidated_info += f"\n--- Batch {i} Details ---\n{resp.choices[0].message.content}\n"
            except Exception as e:
                consolidated_info += f"\n[Error extracting Batch {i}: {str(e)}]\n"
    else:
        consolidated_info = mom_text

    # 2. Draft Generation
    print("[+] Generating SOW Draft...")
    system_prompt = "You are a professional Project Manager and Technical Writer."
    user_prompt = (
        f"I have the following Consolidated Project Details (extracted from meeting minutes):\n"
        f"---------------------\n"
        f"{consolidated_info}\n"
        f"---------------------\n\n"
        f"Instructions:\n"
        f"1. Create a detailed Statement of Work (SOW) draft.\n"
        f"2. Use Markdown formatting (## Headers, - Bullet points).\n"
        f"3. Include standard SOW sections: Project Overview, Scope of Work, Deliverables, Timeline, Pricing/Budget, Governance/Team.\n"
        f"4. Do NOT use HTML tags yet. Just structure the content clearly.\n"
    )
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error Generating SOW Draft: {str(e)}"

def format_sow_to_html(edited_text, template_text, client, model):
    """Wraps edited SOW text in HTML structure of the template."""
    print("[+] Applying Template Formatting...")
    
    system_prompt = "You are a specialized document formatter."
    user_prompt = (
        f"I have the final SOW Content (Markdown):\n"
        f"---------------------\n"
        f"{edited_text}\n"
        f"---------------------\n\n"
        f"And the Style/Structure extracted from a Reference PDF via OCR/Text Extraction:\n"
        f"---------------------\n"
        f"{template_text}\n"
        f"---------------------\n\n"
        f"Instructions:\n"
        f"1. Convert the 'SOW Content' into an HTML document.\n"
        f"2. Mimic the structure and specific standard clauses found in the 'Reference PDF' where applicable, but keep the specific project details from 'SOW Content'.\n"
        f"3. Use HTML tags (<h1>, <p>, <ul> etc.).\n"
        f"4. Output ONLY the valid HTML. Do not include markdown code blocks.\n"
    )
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1, 
        )
        content = response.choices[0].message.content
        content = content.replace("```html", "").replace("```", "").strip()
        return content
    except Exception as e:
        return f"<h3>Error Formatting SOW</h3><p>{str(e)}</p>"

In [ ]:
# @title 3. Configuration & Setup
LLM_BASE_URL = "http://localhost:11434/engines/v1" # @param {type:"string"}
LLM_MODEL = "llama3.2" # @param {type:"string"}
LLM_API_KEY= "docker"
TEMPLATE_FILENAME = "SOW-TEMPLATE.pdf" # @param {type:"string"}

# Retrieve API Key securely
try:
    from google.colab import userdata
    LLM_API_KEY = userdata.get('LLM_API_KEY')
except ImportError:
    import getpass
    print("Enter your LLM API Key:")
    LLM_API_KEY = getpass.getpass()
except Exception:
    import getpass
    print("Could not retrieve key from userData. Enter your LLM API Key:")
    LLM_API_KEY = getpass.getpass()

if not LLM_API_KEY:
    raise ValueError("API Key is required!")

# Initialize Client
client = OpenAI(base_url=LLM_BASE_URL, api_key=LLM_API_KEY)
print(f"✅ Client initialized for {LLM_MODEL}")

# Check Template
if not os.path.exists(TEMPLATE_FILENAME):
    print(f"⚠️ WARNING: '{TEMPLATE_FILENAME}' not found in the current directory.")
    print("Please upload the template PDF to the Files sidebar.")
    # Implement upload fallback if needed, or just warn
    from google.colab import files
    print("Upload template now?")
    uploaded = files.upload()
    if TEMPLATE_FILENAME in uploaded:
        print(f"✅ {TEMPLATE_FILENAME} uploaded.")
    else:
        print("❌ Template upload failed or name mismatch.")

In [ ]:
# @title 4. Input Meeting Minutes
# @markdown Run this cell and enter the MOM text when prompted.
print("Paste your Meeting Minutes below and press Ctrl+D (Linux/Mac) or Ctrl+Z (Windows) followed by Enter to save:")
# Using standard input for multi-line paste in scripts, but in Colab input() is often better for short text. 
# For long text blocks in Colab, forms are better.
MOM_TEXT = """
[PASTE YOUR MEETING MINUTES HERE IN THE CODE CELL IF PREFERRED OR USE INPUT BELOW]
Project: New Web App
Scope: Create a login page and dashboard.
Timeline: 2 weeks.
""" 
# In a real Colab cell, users can edit the string above directly.
# Or use input()
# MOM_TEXT = input("Paste MOM here (single line) or modify the MOM_TEXT variable in the code cell:")

In [ ]:
# @title 5. Generate SOW Draft
if not MOM_TEXT or "PASTE" in MOM_TEXT:
    print("⚠️ Please update the MOM_TEXT variable in standard text or provide input.")
else:
    draft_sow = generate_sow_draft(MOM_TEXT, client, LLM_MODEL)
    print("\n" + "="*40)
    print("GENERATED DRAFT SOW")
    print("="*40)
    print(draft_sow)
    print("="*40)

In [ ]:
# @title 6. Edit Draft (Optional)
# @markdown Modify the `draft_sow` variable below if you want to make changes before generating the PDF.
FINAL_DRAFT_SOW = draft_sow 
# In Colab, the user would interactively edit the cell logic or use a form. 
# We'll assume they proceed with FINAL_DRAFT_SOW.

In [ ]:
# @title 7. Generate PDF
if os.path.exists(TEMPLATE_FILENAME):
    template_content = extract_text_from_pdf(TEMPLATE_FILENAME)
    html_out = format_sow_to_html(FINAL_DRAFT_SOW, template_content, client, LLM_MODEL)
    pdf_bytes = generate_pdf_from_html(html_out)
    
    output_filename = "Generated_SOW.pdf"
    with open(output_filename, "wb") as f:
        f.write(pdf_bytes)
    
    print(f"✅ PDF Generated: {output_filename}")
    
    from google.colab import files
    files.download(output_filename)
else:
    print("❌ Template not found. Cannot generate formatted PDF.")